# TP3 Ex3 Grupo 24
Gabriel Antunes a101101<br>
Guilherme Pinho a105533

Considere de novo o 1º problema do trabalho TP2  relativo à descrição da cifra $$\,\mathsf{A5/1}$$ e o FOTS usando BitVec’s que aí foi definido para a componente do gerador de chaves. Ignore a componente de geração final da chave e restrinja o modelo aos três LFSR’s. 
Sejam $$\,\mathsf{X}_0, \mathsf{X}_1, \mathsf{X}_2\,$$ as variáveis que determinam os estados dos três LFSR’s que ocorrem neste modelo. Como condição inicial  e condição de erro use os predicados

I≡(X​0>0)∧(X​1>0)∧(X​2>0) e $$\quad \mathsf{E}\;\equiv\;\neg\,\mathsf{I}$$

Codifique em “z3”  o SFOTS assim definido.

In [ ]:
from pysmt.shortcuts import * 
from pysmt.typing import BVType

import itertools 

bit_size0 = 19
bit_size1 = 22
bit_size2 = 23

def genState(vars,s,i):
    state = {}
    for v in vars:
        if v == 'X0':
            state[v] = Symbol(v+'!'+s+str(i), types.BVType(19))
        elif v == 'X1':
            state[v] = Symbol(v+'!'+s+str(i), types.BVType(22))
        else:
            state[v] = Symbol(v+'!'+s+str(i), types.BVType(23))
    return state

def init(s):
    return And(
        BVUGT(s['X0'], BV(0, 19)),   
        BVUGT(s['X1'], BV(0, 22)),
        BVUGT(s['X2'], BV(0, 23)) 
        )

def error(s):
    return Or(
        BVULT(s['X0'], BV(0, 19)),  
        BVULT(s['X1'], BV(0, 22)),  
        BVULT(s['X2'], BV(0, 23))   
    )

# Bits de Controlo
def controlBits(a, b, c):
    return Or(Equals(a,b), Equals(a,c))

# Transição global combinando os três LFSRs
def trans(atual, prox):

    s0 = BV(466944,19)
    s1 = BV(3145728,22)
    s2 = BV(7340160,23)
    
    b0 = BVExtract(atual['X0'], 8, 8)
    b1 = BVExtract(atual['X1'], 10, 10)
    b2 = BVExtract(atual['X2'], 10, 10)

    t0 = And(controlBits(b0, b1, b2), controlBits(b1, b0, b2), controlBits(b2, b0, b1),
             Equals(prox['X0'], BVXor(BVLShl(atual['X0'], BV(1, 19)), BVXor(s0, atual['X0']))),
             Equals(prox['X1'], BVXor(BVLShl(atual['X1'], BV(1, 22)), BVXor(s1, atual['X1']))),
             Equals(prox['X2'], BVXor(BVLShl(atual['X2'], BV(1, 23)), BVXor(s2, atual['X2'])))
    )

    t1 = And(Not(controlBits( b0, b1, b2)), controlBits( b1, b0, b2), controlBits( b2, b0, b1),
            Equals(prox['X1'], BVXor(BVLShl(atual['X1'], BV(1, 22)), BVXor(s1, atual['X1']))),
            Equals(prox['X2'], BVXor(BVLShl(atual['X2'], BV(1, 23)), BVXor(s2, atual['X2']))),
            Equals(prox['X0'], atual['X0'])
    )

    t2 = And(controlBits( b0, b1, b2), Not(controlBits( b1, b0, b2)), controlBits( b2, b0, b1),
            Equals(prox['X0'], BVXor(BVLShl(atual['X0'], BV(1, 19)), BVXor(s0, atual['X0']))),
            Equals(prox['X2'], BVXor(BVLShl(atual['X2'], BV(1, 23)), BVXor(s2, atual['X2']))),
            Equals(prox['X1'], atual['X1'])
    )

    t3 = And(controlBits( b0, b1, b2), controlBits( b1, b0, b2), Not(controlBits( b2, b0, b1)),
             Equals(prox['X0'], BVXor(BVLShl(atual['X0'], BV(1, 19)), BVXor(s0, atual['X0']))),
             Equals(prox['X1'], BVXor(BVLShl(atual['X1'], BV(1, 22)), BVXor(s1, atual['X1']))),
             Equals(prox['X2'], atual['X2'])
    )

    return Or(t0, t1, t2, t3)

def genTrace(vars, init, trans, error, n):
    
    with Solver(name="z3") as solver:
        X = [genState(vars, 'X', i) for i in range(n+1)]     # cria n+1 estados (com etiqueta X)

        I = init(X[0]) 
        Tks = [trans(X[i], X[i+1]) for i in range(n)] 
        
        if solver.solve([I, And(Tks)]): 
            for i in range(n+1):
                print(f"Estado {i}:")
                for v in X[i]:
                    print(f"  {v} = {solver.get_value(X[i][v])}")
        else:
            print("Nenhum traço válido encontrado.")
            
vars = ['X0','X1','X2']

genTrace(vars, init, trans, error, 2) 

Estado 0:
  X0 = 174762_19
  X1 = 3565158_22
  X2 = 1887411_23
Estado 1:
  X0 = 174762_19
  X1 = 2796202_22
  X2 = 5592405_23
Estado 2:
  X0 = 57342_19
  X1 = 1048574_22
  X2 = 5592405_23


Use o algoritmo PDR “property directed reachability” (codifique-o ou use uma versão pré-existente) e, com ele, tente provar a segurança deste modelo.

In [16]:
def pdr(vars, init, trans, error, iterations):
    with Solver(name="z3") as solver:

        # Lista de estados simbólicos para os frames
        X = [init(genState(vars, 'F', 0))]  # Estado inicial com restrições de inicialização

        for k in range(iterations):
            # 1. Expandir o frame atual excluindo estados de erro
            curr = genState(vars, 'F', k)
            newS = And(X[-1], Not(error(curr)))

            # Verificar se o novo frame exclui estados de erro
            if not solver.solve([newS, error(curr)]):
                print(f"Propriedade garantida no Estado {k}.")
                return True

            # Adicionar o novo frame à lista de frames
            X.append(newS)

            # 2. Indução: verificar transições entre o estado atual e próximo estado
            for i in range(k):
                curr = genState(vars, 'F', i)
                nxt = genState(vars, 'F', i+1)

                # Verificar se os estados em i são invariantes
                if solver.solve([X[i], trans(curr, nxt), Not(X[i + 1])]):
                    print(f"Invariante falhou entre frames {i} e {i+1}.")
                    return False

        print("Propriedade não foi provada. Limite de iterações atingido.")
        return False


# Testar o algoritmo
vars = ['X0', 'X1', 'X2']
pdr(vars, init, trans, error, 10)


Propriedade garantida no Estado 0.


True